#Installations

In [ ]:
!pip install datasets

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate>=0.21.0

Restart session after running all pip commands

#Imports

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import default_data_collator, Trainer
import numpy as np
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import collections
from tqdm.auto import tqdm

#Load saved model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load model directly
model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/Hindi_Tamil_QnA/chaii-bert-trained")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-large-squad2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

#Processing Functions and parameters

In [27]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [28]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []

        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [29]:
batch_size = 4
max_length = 384
doc_stride = 128
pad_on_right = tokenizer.padding_side == "right"
max_answer_length = 30

#Fetch Data

In [30]:
# !unzip /content/drive/MyDrive/Hindi_Tamil_QnA/chaii-hindi-and-tamil-question-answering.zip

In [31]:
# df_test= pd.read_csv("/content/test.csv")

**C1**

निफ्टी या निफ्टी50, नेशनल स्टॉक एक्सचेंज (एनएसई) का एक प्रमुख सूचकांक है, जो 13 प्रमुख क्षेत्रों से एनएसई पर कारोबार करने वाली शीर्ष 50 कंपनियों का प्रतिनिधित्व करता है। यह सूचकांक निवेशकों को बाजार की भावनाओं और प्रदर्शन पर एक विहंगम दृष्टि देता है। इसलिए, इसे बड़े पैमाने पर भारतीय शेयर बाजार और अर्थव्यवस्था का सच्चा प्रतिबिंब माना जाता है। इसके कारण, कई लोग इस सूचकांक के प्रदर्शन को एक अवधि में अपने पोर्टफोलियो के प्रदर्शन के मुकाबले एक बेंचमार्क मानते हैं।
इसे 22 अप्रैल 1996 को लॉन्च किया गया था, यानी इस साल यह 25 साल का हो गया! 22 अप्रैल, 1996 को 1,107 से 15 जून 2021 को 15,901.60 की नई रिकॉर्ड ऊंचाई तक इसने कितनी अद्भुत यात्रा देखी है।


**Q1**

निफ्टी50 क्या है?

**C2**

राउज एवेन्यू कोर्ट द्वारा आम आदमी पार्टी (आप) के संयोजक को 15 अप्रैल तक न्यायिक हिरासत में भेजने का आदेश दिए जाने के बाद दिल्ली के मुख्यमंत्री अरविंद केजरीवाल को तिहाड़ जेल ले जाया गया। कई आप नेताओं ने तिहाड़ जेल के बाहर विरोध प्रदर्शन जारी रखा। दिल्ली के मुख्यमंत्री अरविंद केजरीवाल को न्यायिक हिरासत में भेजे जाने के बाद उनकी पत्नी सुनीता केजरीवाल ने केंद्र की आलोचना करते हुए कहा कि "देश की जनता इस तानाशाही का जवाब देगी।" सुनीता केजरीवाल ने पूछा, "अगर जांच पूरी हो गई थी तो उन्हें जेल क्यों भेजा गया? देश की जनता इस तानाशाही का जवाब देगी।" इससे पहले, अदालत ने 28 मार्च को केजरीवाल की ईडी हिरासत 1 अप्रैल तक बढ़ा दी थी। प्रवर्तन निदेशालय ने कथित शराब नीति मामले में 21 मार्च को आप सुप्रीमो को गिरफ्तार किया था।

**Q2**

अरविंद केजरीवाल कौन हैं?

**C3**

राजा कृष्णदेव राय के राज्य में चेलाराम नाम का एक व्यक्ति रहता था। वह राज्य में इस बात से प्रसिद्ध था कि अगर कोई सुबह-सवेरे उसका चेहरा सबसे पहले देख ले तो उसे दिनभर खाने को कुछ नहीं मिलता। लोग उसे मनहूस कहकर पुकारते थे। बेचारा चेलाराम इस बात से दुखी तो होता, लेकिन फिर भी अपने काम में लगा रहता। एक दिन यह बात राजा के कानों तक जा पहुंची। राजा इस बात को सुनकर बहुत उत्सुक हुए। वह जानना चाहते थे कि क्या चेलाराम सच में इतना मनहूस है? अपनी इस उत्सुकता को दूर करने के लिए उन्होंने चेलाराम को महल में हाजिर होने का बुलावा भेजा। दूसरी ओर चेलारम इस बात से अंजान खुशी-खुशी महल के लिए चल पड़ा। महल पहुंचने पर जब राजा ने उसे देखा तो वे सोचने लगे कि यह चेलारम तो दूसरों की भांति सामान्य प्रतीत होता है। यह कैसे दूसरे लोगों के लिए मनहूसियत का कारण हो सकता है। इस बात को परखने के लिए उन्होंने आदेश दिया कि चेलाराम को उनके शयनकक्ष के सामने वाले कमरे में ठहराया जाए।

**Q3**

चेलाराम क्यों प्रसिद्ध थे?

In [34]:
n=3
id=[]
context=[]
question=[]
language=[]

for i in range(n):
  id1 =input("Enter ID: ")
  id.append(id1)
  context1 = input("Enter Context: ")
  context.append(context1)
  question1 = input("Enter Question: ")
  question.append(question1)
  language1 = input("Enter Language: ")
  language.append(language1)

Enter ID: 1
Enter Context:  निफ्टी या निफ्टी50, नेशनल स्टॉक एक्सचेंज (एनएसई) का एक प्रमुख सूचकांक है, जो 13 प्रमुख क्षेत्रों से एनएसई पर कारोबार करने वाली शीर्ष 50 कंपनियों का प्रतिनिधित्व करता है। यह सूचकांक निवेशकों को बाजार की भावनाओं और प्रदर्शन पर एक विहंगम दृष्टि देता है। इसलिए, इसे बड़े पैमाने पर भारतीय शेयर बाजार और अर्थव्यवस्था का सच्चा प्रतिबिंब माना जाता है। इसके कारण, कई लोग इस सूचकांक के प्रदर्शन को एक अवधि में अपने पोर्टफोलियो के प्रदर्शन के मुकाबले एक बेंचमार्क मानते हैं। इसे 22 अप्रैल 1996 को लॉन्च किया गया था, यानी इस साल यह 25 साल का हो गया! 22 अप्रैल, 1996 को 1,107 से 15 जून 2021 को 15,901.60 की नई रिकॉर्ड ऊंचाई तक इसने कितनी अद्भुत यात्रा देखी है।
Enter Question: निफ्टी50 क्या है?
Enter Language: hindi
Enter ID: 2
Enter Context: राउज एवेन्यू कोर्ट द्वारा आम आदमी पार्टी (आप) के संयोजक को 15 अप्रैल तक न्यायिक हिरासत में भेजने का आदेश दिए जाने के बाद दिल्ली के मुख्यमंत्री अरविंद केजरीवाल को तिहाड़ जेल ले जाया गया। कई आप नेताओं ने तिहाड़ जेल के बाहर विरोध प्रदर्शन जारी 

In [35]:
test_df = pd.DataFrame()

In [36]:
test_df["id"]=id
test_df["context"]=context
test_df["question"]=question
test_df["language"]=language

In [37]:
test_df

,id,context,question,language
0,1,"निफ्टी या निफ्टी50, नेशनल स्टॉक एक्सचेंज (एनए...",निफ्टी50 क्या है?,hindi
1,2,राउज एवेन्यू कोर्ट द्वारा आम आदमी पार्टी (आप) ...,अरविंद केजरीवाल कौन हैं?,hindi
2,3,राजा कृष्णदेव राय के राज्य में चेलाराम नाम का ...,चेलाराम क्यों प्रसिद्ध थे?,hindi


#Prepare Dataset and map features

In [38]:
test_dataset = Dataset.from_pandas(test_df)

test_features = test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [39]:
test_feats_small = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
test_feats_small

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3
})

#Define Trainer and predict

In [40]:
data_collator = default_data_collator

trainer = Trainer(model=model)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [41]:
test_predictions = trainer.predict(test_feats_small)

In [42]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

In [43]:
final_test_predictions = postprocess_qa_predictions(test_dataset, test_features, test_predictions.predictions)

Post-processing 3 example predictions split into 3 features.


  0%|          | 0/3 [00:00<?, ?it/s]

In [44]:
print(final_test_predictions)

OrderedDict([('1', ' नेशनल स्टॉक एक्सचेंज (एनएसई) का एक प्रमुख सूचकांक'), ('2', ' दिल्ली के मुख्यमंत्री'), ('3', ' अगर कोई सुबह-सवेरे उसका चेहरा सबसे पहले देख ले तो उसे दिनभर खाने को कुछ नहीं मिलता')])


In [45]:
test_df['PredictionString'] = test_df['id'].apply(lambda r: final_test_predictions[r])
test_df.head()

,id,context,question,language,PredictionString
0,1,"निफ्टी या निफ्टी50, नेशनल स्टॉक एक्सचेंज (एनए...",निफ्टी50 क्या है?,hindi,नेशनल स्टॉक एक्सचेंज (एनएसई) का एक प्रमुख सूच...
1,2,राउज एवेन्यू कोर्ट द्वारा आम आदमी पार्टी (आप) ...,अरविंद केजरीवाल कौन हैं?,hindi,दिल्ली के मुख्यमंत्री
2,3,राजा कृष्णदेव राय के राज्य में चेलाराम नाम का ...,चेलाराम क्यों प्रसिद्ध थे?,hindi,अगर कोई सुबह-सवेरे उसका चेहरा सबसे पहले देख ल...


In [46]:
test_df.to_csv("results.csv")